In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calcular_admitancia(n_barras, lineas):
    Y = np.zeros((n_barras, n_barras), dtype=complex)
    for linea in lineas:
        i, j, r, x, b = linea
        Z = complex(r, x)
        Y[i-1, j-1] = -1/Z
        Y[j-1, i-1] = Y[i-1, j-1]
    
    for k in range(n_barras):
        Y[k, k] = -np.sum(Y[k, :])
    
    return Y

def flujo_carga_newton_raphson(n_barras, lineas, datos_barras, tol=1e-6, max_iter=20):
    Y = calcular_admitancia(n_barras, lineas)
    P = datos_barras[:, 1]  # Potencia activa especificada (P)
    Q = datos_barras[:, 2]  # Potencia reactiva especificada (Q)
    V = datos_barras[:, 3]  # Voltajes especificados o iniciales (V)
    ang = datos_barras[:, 4]  # Ángulos especificados o iniciales (Theta)

    PQ_indices = np.where(datos_barras[:, 0] == 1)[0]  # Indices de barras PQ
    PV_indices = np.where(datos_barras[:, 0] == 2)[0]  # Indices de barras PV
    slack_index = np.where(datos_barras[:, 0] == 3)[0][0]  # Índice de la barra slack

    n_PQ = len(PQ_indices)
    n_PV = len(PV_indices)

    # Newton-Raphson Iteration
    for iteration in range(max_iter):
        # Calculamos las potencias especificadas
        P_calc = np.zeros(n_barras)
        Q_calc = np.zeros(n_barras)
        
        for i in range(n_barras):
            for k in range(n_barras):
                if i != k:
                    P_calc[i] += V[i] * V[k] * (Y[i, k].real * np.cos(ang[i] - ang[k]) + Y[i, k].imag * np.sin(ang[i] - ang[k]))
                    Q_calc[i] -= V[i] * V[k] * (Y[i, k].real * np.sin(ang[i] - ang[k]) - Y[i, k].imag * np.cos(ang[i] - ang[k]))
            P_calc[i] += V[i]**2 * Y[i, i].real
            Q_calc[i] -= V[i]**2 * Y[i, i].imag

        # Desviaciones de potencia
        dP = P - P_calc
        dQ = Q - Q_calc

        # Construcción de la matriz Jacobiana
        J = np.zeros((2*n_PQ + n_PV, 2*n_PQ + n_PV))
        
        # Submatriz J1 (Derivadas de P respecto a Theta)
        for i in range(n_PQ + n_PV):
            for k in range(n_PQ + n_PV):
                if i == k:
                    J[i, k] = -Q_calc[PQ_indices[i]] - V[PQ_indices[i]]**2 * Y[PQ_indices[i], PQ_indices[i]].imag
                else:
                    J[i, k] = V[PQ_indices[i]] * V[PQ_indices[k]] * (Y[PQ_indices[i], PQ_indices[k]].real * np.sin(ang[PQ_indices[i]] - ang[PQ_indices[k]]) - Y[PQ_indices[i], PQ_indices[k]].imag * np.cos(ang[PQ_indices[i]] - ang[PQ_indices[k]]))

        # Submatriz J2 (Derivadas de P respecto a V)
        for i in range(n_PQ + n_PV):
            for k in range(n_PQ):
                if PQ_indices[i] == PQ_indices[k]:
                    J[i, n_PQ + n_PV + k] = V[PQ_indices[i]] * P_calc[PQ_indices[i]]
                else:
                    J[i, n_PQ + n_PV + k] = V[PQ_indices[i]] * (Y[PQ_indices[i], PQ_indices[k]].real * np.cos(ang[PQ_indices[i]] - ang[PQ_indices[k]]) + Y[PQ_indices[i], PQ_indices[k]].imag * np.sin(ang[PQ_indices[i]] - ang[PQ_indices[k]]))

        # Submatriz J3 (Derivadas de Q respecto a Theta)
        for i in range(n_PQ):
            for k in range(n_PQ + n_PV):
                if PQ_indices[i] == PQ_indices[k]:
                    J[n_PQ + n_PV + i, k] = V[PQ_indices[i]] * Q_calc[PQ_indices[i]]
                else:
                    J[n_PQ + n_PV + i, k] = -V[PQ_indices[i]] * V[PQ_indices[k]] * (Y[PQ_indices[i], PQ_indices[k]].real * np.cos(ang[PQ_indices[i]] - ang[PQ_indices[k]]) + Y[PQ_indices[i], PQ_indices[k]].imag * np.sin(ang[PQ_indices[i]] - ang[PQ_indices[k]]))

        # Submatriz J4 (Derivadas de Q respecto a V)
        for i in range(n_PQ):
            for k in range(n_PQ):
                if i == k:
                    J[n_PQ + n_PV + i, n_PQ + n_PV + k] = -P_calc[PQ_indices[i]] + V[PQ_indices[i]]**2 * Y[PQ_indices[i], PQ_indices[i]].real
                else:
                    J[n_PQ + n_PV + i, n_PQ + n_PV + k] = -V[PQ_indices[i]] * (Y[PQ_indices[i], PQ_indices[k]].real * np.sin(ang[PQ_indices[i]] - ang[PQ_indices[k]]) - Y[PQ_indices[i], PQ_indices[k]].imag * np.cos(ang[PQ_indices[i]] - ang[PQ_indices[k]]))

        # Vector de desviaciones
        dPQ = np.concatenate((dP[PQ_indices], dQ[PQ_indices]))

        # Solución del sistema lineal
        dx = np.linalg.solve(J, dPQ)

        # Actualización de ángulos y tensiones
        ang[PQ_indices] += dx[:n_PQ + n_PV]
        V[PQ_indices] += dx[n_PQ + n_PV:]

        # Verificación de convergencia
        if np.max(np.abs(dx)) < tol:
            break

    return V, ang

#Definición de las líneas de transmisión
# Cada línea está definida por:
# [Barra de origen (i), Barra de destino (j), Resistencia (pu), Reactancia (pu), Susceptancia (pu)]
n_barras = 6
lineas = [
    [1, 2, 0.02, 0.04, 0.01],
    [2, 3, 0.01, 0.03, 0.01],
    [3, 4, 0.02, 0.05, 0.01],
    [4, 5, 0.01, 0.03, 0.01],
    [5, 6, 0.03, 0.06, 0.01]
]
# Definición de los datos de las barras
# Cada fila representa una barra con los siguientes datos:
# [Tipo de barra, P especificada (pu), Q especificada (pu), Voltaje especificado (pu), Ángulo especificado (radianes)]
datos_barras = np.array([
    [3, 0.0, 0.0, 1.06, 0.0],  # Barra 1 (slack)
    [2, -1.0, 0.0, 1.0, 0.0],  # Barra 2 (PV)
    [1, -1.5, -0.5, 1.0, 0.0],  # Barra 3 (PQ)
    [1, -0.6, -0.2, 1.0, 0.0],  # Barra 4 (PQ)
    [1, -0.8, -0.3, 1.0, 0.0],  # Barra 5 (PQ)
    [1, -0.9, -0.4, 1.0, 0.0]   # Barra 6 (PQ)
])

# Resolver flujo de carga
V, ang = flujo_carga_newton_raphson(n_barras, lineas, datos_barras)

# Mostrar resultados
for i in range(n_barras):
    print(f'Barra {i+1}: V = {V[i]:.4f} pu, Theta = {ang[i]:.4f} rad')

# Graficar tensiones
plt.figure()
plt.bar(range(1, n_barras+1), V)
plt.xlabel('Barra')
plt.ylabel('Tensión (pu)')
plt.title('Tensiones en las barras')
plt.grid(True)
plt.show()

# Graficar ángulos
plt.figure()
plt.bar(range(1, n_barras+1), ang)
plt.xlabel('Barra')
plt.ylabel('Ángulo (rad)')
plt.title('Ángulos en las barras')
plt.grid(True)
plt.show()

# Calcular corrientes de línea
I_lines = []
for linea in lineas:
    i, j, r, x, _ = linea
    Z = complex(r, x)
    V_i = V[i-1] * (np.cos(ang[i-1]) + 1j*np.sin(ang[i-1]))
    V_j = V[j-1] * (np.cos(ang[j-1]) + 1j*np.sin(ang[j-1]))
    I_line = (V_i - V_j) / Z
    I_lines.append(np.abs(I_line))

# Graficar corrientes de línea
plt.figure()
plt.bar(range(1, len(I_lines)+1), I_lines)
plt.xlabel('Línea')
plt.ylabel('Corriente (pu)')
plt.title('Corrientes en las líneas')
plt.grid(True)
plt.show()

